## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-03-08-47-00 +0000,wsj,Swiss Inflation Declines as SNB Mulls Negative...,https://www.wsj.com/economy/swiss-inflation-de...
1,2025-11-03-08-40-53 +0000,nypost,Powerful earthquake kills at least 20 people i...,https://nypost.com/2025/11/03/world-news/earth...
2,2025-11-03-08-38-50 +0000,wsj,Stock Market Today: S&P 500 Futures Inch Up; G...,https://www.wsj.com/livecoverage/stock-market-...
3,2025-11-03-08-38-00 +0000,wsj,Sterling Falls as Rate-Cut Bets Rise Ahead of ...,https://www.wsj.com/finance/currencies/asian-c...
4,2025-11-03-08-35-00 +0000,wsj,BP to Sell Stakes in U.S Midstream Assets for ...,https://www.wsj.com/business/energy-oil/bp-to-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
57,trump,26
72,new,10
122,will,9
323,mamdani,8
742,train,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
51,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...,69
119,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...,68
144,2025-11-02-17-00-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...,63
59,2025-11-03-01-32-00 +0000,wsj,Democratic senators again urged President Trum...,https://www.wsj.com/politics/policy/democrats-...,55
174,2025-11-02-11-26-41 +0000,nypost,White House unveils details of ‘historic’ trad...,https://nypost.com/2025/11/02/us-news/donald-t...,53


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
51,69,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...
163,47,2025-11-02-14-11-04 +0000,nypost,UK pol tells country to ‘get on with your live...,https://nypost.com/2025/11/02/world-news/uk-po...
119,42,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...
144,37,2025-11-02-17-00-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...
145,35,2025-11-02-16-46-09 +0000,nypost,House Dem leader rails that Mamdani isn’t futu...,https://nypost.com/2025/11/02/us-news/house-de...
150,33,2025-11-02-16-11-31 +0000,bbc,"'No help, no food, no water': Hurricane-hit Ja...",https://www.bbc.com/news/articles/cpv11w8ez10o...
194,29,2025-11-02-10-00-00 +0000,wsj,"Republican Jack Ciattarelli, locked in a tight...",https://www.wsj.com/politics/elections/new-jer...
94,28,2025-11-02-21-52-09 +0000,nyt,"For Day of the Dead, One Mexican Town Digs Up ...",https://www.nytimes.com/2025/11/02/world/ameri...
55,25,2025-11-03-02-00-00 +0000,wsj,Even if Democrats have election wins on Tuesda...,https://www.wsj.com/politics/elections/electio...
131,25,2025-11-02-18-36-44 +0000,latimes,Photos: Los Angeles celebrates Dodgers' first ...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
